In [1]:
import requests
import re
import json
import pandas as pd
import time
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

In [2]:
# 1. Obtener el HTML de la página del ranking
load_dotenv()
url = os.getenv("TARGET_URL_GDP")
BASE_URL = os.getenv("BASE_URL")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Lanza error si la petición falla

In [3]:
# 2. Parsear HTML con BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
# 3. Buscar la tabla principal y extraer los links
table = soup.find("table")
country_links = []


if table:
    rows = table.find_all("tr")
    for row in rows:
        cell = row.find("td")
        if cell and cell.a:
            relative_link = cell.a["href"]
            full_url = f"{BASE_URL}{relative_link}"
            country_links.append(full_url)

# Convertir la lista en DataFrame
df_links = pd.DataFrame(country_links, columns=["url"])

# Guardar como archivo Parquet
df_links.to_parquet("data/country_gdp_links.parquet", index=False)

In [5]:
# 4. Mostrar resultado
print("Links encontrados:", df_links.size)

Links encontrados: 125


In [6]:
# configuracion necesaria para descargar datos

master_df = pd.DataFrame()

pattern = r'arrayToDataTable\((\[\[.*?\]\])\)'

sleep_duration = 5

In [7]:
# Cargar los países válidos desde el archivo parquet
df_countries = pd.read_parquet("data/countries_with_hpi.parquet")
valid_countries = set(df_countries["country"].unique())

In [8]:
master_df.to_parquet("gdp_values.parquet", index=False)

In [9]:
start_time = time.time()
for url in country_links:
    # Extraer país desde el URL
    country = urlparse(url).path.strip("/").split("/")[0]

    # Verificar si el país está en la lista válida
    if country not in valid_countries:
        print(f"⏭️ País {country} no está en la lista, se omite.")
        continue

    try:
        response = requests.get(url, headers=headers)
        html = response.text

        match = re.search(pattern, html, re.DOTALL)
        if match:
            data_str = match.group(1)
            data = json.loads(data_str)

            df = pd.DataFrame(data[1:], columns=data[0])
            df.columns = ["Period", "Value"]
            df["Country"] = country

            df_pivot = df.pivot(index="Period", columns="Country", values="Value")
            master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

            print(f"✅ Datos extraídos correctamente para {country}")
        else:
            print(f"⚠️ No se encontraron datos en {url}")
    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")
    
    time.sleep(sleep_duration)

end_time = time.time()
total_sleep_time = len(valid_countries) * sleep_duration
effective_time = end_time - start_time - total_sleep_time

print(f"⏳ Tiempo total (incluyendo esperas): {end_time - start_time:.2f} s")
print(f"😴 Tiempo en espera: {total_sleep_time:.2f} s")
print(f"⚡ Tiempo efectivo de ejecución: {effective_time:.2f} s")

⏭️ País Afghanistan no está en la lista, se omite.
⏭️ País Albania no está en la lista, se omite.
⏭️ País Algeria no está en la lista, se omite.
⏭️ País Argentina no está en la lista, se omite.
⏭️ País Armenia no está en la lista, se omite.
✅ Datos extraídos correctamente para Australia
✅ Datos extraídos correctamente para Austria
⏭️ País Azerbaijan no está en la lista, se omite.
⏭️ País Bahamas no está en la lista, se omite.
⏭️ País Bahrain no está en la lista, se omite.
⏭️ País Bangladesh no está en la lista, se omite.
⏭️ País Belarus no está en la lista, se omite.
✅ Datos extraídos correctamente para Belgium
⏭️ País Bermuda no está en la lista, se omite.
⏭️ País Bolivia no está en la lista, se omite.
⏭️ País Bosnia-and-Herzegovina no está en la lista, se omite.
⏭️ País Botswana no está en la lista, se omite.
✅ Datos extraídos correctamente para Brazil
⏭️ País Brunei no está en la lista, se omite.
✅ Datos extraídos correctamente para Bulgaria
⏭️ País Burkina-Faso no está en la lista,

In [10]:
master_df.head()

,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
Period,,,,,,,,,,,,,,,,,,,,,
Q1 1960,3.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,49.06,NaN,NaN,NaN,NaN,NaN,135.83,6.43
Q1 1961,4.23,NaN,NaN,NaN,NaN,9.51,NaN,NaN,NaN,NaN,...,NaN,NaN,61.95,NaN,NaN,NaN,NaN,NaN,136.48,6.88
Q1 1962,4.27,NaN,NaN,NaN,NaN,10.31,NaN,NaN,NaN,NaN,...,NaN,NaN,72.67,NaN,NaN,NaN,NaN,NaN,148.80,7.13
Q1 1963,4.66,NaN,NaN,NaN,NaN,11.01,NaN,NaN,NaN,NaN,...,NaN,NaN,97.56,NaN,NaN,NaN,NaN,NaN,155.68,7.37
Q1 1964,5.06,NaN,NaN,NaN,NaN,12.29,NaN,NaN,NaN,NaN,...,NaN,NaN,128.53,NaN,NaN,NaN,NaN,NaN,167.78,8.14


In [11]:
# obtain paraguay url
template_url = country_links[0]
country = "Paraguay"

# Separar por "/"
parts = template_url.strip("/").split("/")

# Reemplazar el país (posición 3: después del dominio)

parts[3] = country

# Reconstruir la URL
custom_url = "/".join(parts) + "/"

In [12]:
# extract and include paraguay

response = requests.get(custom_url, headers=headers)
html = response.text

match = re.search(pattern, html, re.DOTALL)
if match:
    data_str = match.group(1)
    data = json.loads(data_str)

    df = pd.DataFrame(data[1:], columns=data[0])
    df.columns = ["Period", "Value"]
    df["Country"] = country

    df_pivot = df.pivot(index="Period", columns="Country", values="Value")
    master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

    print(f"✅ Datos extraídos correctamente para {country}")

✅ Datos extraídos correctamente para Paraguay


In [13]:
master_df.tail()

,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
Period,,,,,,,,,,,,,,,,,,,,,
Q4 2020,531.33,100.88,126.79,2059.46,34.11,595.92,56687.99,29561.88,275280.92,12.98,...,1499.66,541604.8,302.40,1345.44,180.40,4117.57,1525.15,5369.40,558.51,66436.69
Q4 2021,589.36,111.52,139.22,2355.98,40.27,677.29,67065.22,32589.94,337508.40,14.90,...,1633.40,589685.1,336.72,1481.90,194.12,4301.57,2331.62,6087.28,612.76,72468.28
Q4 2022,662.03,119.48,153.61,2641.49,49.70,729.12,71762.50,34234.24,394397.85,17.24,...,1729.16,599728.8,368.10,1564.45,202.82,4545.88,4794.18,6602.10,668.15,78219.41
Q4 2023,690.21,122.28,158.09,2865.30,53.18,764.41,75383.79,35722.48,415041.45,19.52,...,1840.02,635747.1,396.07,1626.25,204.08,4638.67,8527.68,7074.25,693.21,83109.55
Q4 2024,715.14,126.77,163.78,3080.37,57.59,798.84,84662.46,37372.62,461674.90,21.25,...,1928.18,668933.8,420.44,1698.81,210.09,4823.23,12704.05,7430.98,735.14,89086.69


In [14]:
# Resetear el índice para separar "Period"
master_df = master_df.reset_index()

# Separar "Q1 1990" en "Quarter" y "Year"
master_df[["Quarter", "Year"]] = master_df["Period"].str.extract(r"Q(\d)\s+(\d{4})").astype(int)

# Reordenar las columnas: primero Quarter y Year
cols = ["Quarter", "Year"] + [col for col in master_df.columns if col not in ["Period", "Quarter", "Year"]]
master_df = master_df[cols]

# Ordenar por Year y Quarter
master_df = master_df.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [15]:
master_df.tail()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
257,2,2024,678.35,120.93,154.65,2921.23,48.14,753.92,75091.54,32883.76,...,1827.69,636205.4,400.31,1618.85,206.50,4520.55,9920.84,7254.18,699.17,81953.81
258,3,2024,676.79,119.00,147.88,2989.91,53.77,800.82,75344.20,34175.80,...,1859.09,643531.6,393.31,1548.65,206.60,4615.84,11915.59,7343.73,716.05,80770.75
259,4,2024,715.14,126.77,163.78,3080.37,57.59,798.84,84662.46,37372.62,...,1928.18,668933.8,420.44,1698.81,210.09,4823.23,12704.05,7430.98,735.14,89086.69
260,1,2025,675.42,119.38,152.24,3019.58,45.62,757.58,82042.34,31875.80,...,1798.80,622771.1,397.66,1552.81,207.83,4744.03,12125.17,7490.50,744.73,92478.27
261,2,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,423.04,NaN,NaN,NaN,NaN,7582.78,NaN,NaN


In [17]:
master_df.to_parquet("data/gdp_values.parquet", index=False)